Source Code Tugas Pemograman 1 Kelompok 15 

---
Anggota:

*   Alif Adwitiya Pratama
*   Sabrina Adinda Sari
*Wana Ardilah Iwan 








# Library yang akan kita gunakan

In [ ]:
import numpy as np
from random import randint
from numpy import random
import math
import struct
from random import choices
np.set_printoptions(formatter={'float': lambda x: "{0:0.20f}".format(x)})
import pandas as pd


# Fungsi Fitness yang akan digunakan

In [ ]:

#fungsi fitness utama
def fitness(x,y):                                      #input = 2 nilai dengan x antara -1  -  2      dan  y antara -1  -  1  
  if -1 <= x <= 2 and -1 <= y <= 1:                    #output = nilai hasil fungsi
    return (math.cos(x**2)*math.sin(y**2))+(x+y)
  else:
    return np.nan


#dibuat untuk melakukan iterasi dan memasukannya ke fungsi fitness utama
def getFitness(value):                                                        #input: array populasi yang berisi list 2 float
  hasil = np.empty((0, 1), float)                                             #output: mendapatkan nilai hasil fungsi dengan memanfaatkan fungsi fitness
  for ind in value:     
      hasil=np.insert(hasil,0,fitness(ind[0],ind[1]),axis=0)
  hasil=np.flip(hasil)

  return hasil



# Generate Individu

In [ ]:
def generateIndividu(banyak):                                                  #input = banyak individu
    population = np.empty((0, 2), float)                                       #output = array populasi yang berisi list 2 float

    for i in range(0,banyak):
      population=np.insert(population,0,[round(random.uniform(-1.0, 2.0), 8),round(random.uniform(-1.0, 1.0), 8)], axis=0)
    return population


# Encode dan Decode 

In [ ]:

#fungsi encode utama
def encode(value):    # float -> bin                                                 #input = nilai float
    getBin = lambda x: x > 0 and str(bin(x))[2:] or "-" + str(bin(x))[3:]            #output = representasi biner ieee 754 64 bit float point
    val = struct.unpack('Q', struct.pack('d', value))[0]
    ans=str(getBin(val))   
    if len(ans)==62:
      return"00"+ans
    if len(ans)==63:
      return"0"+ans
    return ans

#fungsi decode utama
def decode(value):     #bin -> float                                            #input = representasi biner ieee 754 64 bit float point
    if value[0]=='1':                                                           #output = nilai float
      hx = hex(int(value[1:], 2))  
      return -1*(struct.unpack("d", struct.pack("q", int(hx, 16)))[0])
    else:
      hx = hex(int(value, 2)) 
      return (struct.unpack("d", struct.pack("q", int(hx, 16)))[0])

#melakukan iterasi dan memasukannya kedalam fungsi encode utama 
def encodeList(value):                                                        #input = list float value
  value=value.tolist()                                                        #output = dengan memanfaatkan fungsi encode melakukan encoding nilai satu persatu list menjadi biner

  for i in range(0,len(value)):
    for j in range(0,len(value[i])):
      value[i][j]=encode(value[i][j])
  return value

#melakukan iterasi dan memasukannya kedalam fungsi decode utama
def decodeList(value):                                                        #input = list representasi biner ieee 754 64 bit float point
  result = np.empty((0, 2), float)                                            #output = dengan memanfaatkan fungsi decode melakukan decidubg nilai satu persatu list menjadi float
  for i in range(0,len(value)):
    for j in range(0,len(value[i])):
      result=np.insert(result,0,decode(value[i][j]),axis=0)
  return result


# Seleksi Orangtua

In [ ]:
def makeProb(fit):                                                               #input : nilai fitness
    prob=np.empty((0, 1), float)                                                 #output: nilai probabilitas dari masing masing orang tua
    for v in fit:
      temp=v/np.sum(fit)
      prob=np.insert(prob,0,temp,axis=0)
    prob=np.flip(prob)
    return prob

def seleksiOrangtua(populasi,jumAnak):                                          #input : array populasi dan banyak anak yang diinginkan
    fit=getFitness(populasi)                                                    #output : array populasi orang tua yang akan di cross over 
    prob=makeProb(fit)
    diambil = np.empty((0, 2), float)
    for i in range(0,jumAnak):
      diambil=np.insert(diambil,0,choices(populasi, prob)[0],axis=0)

    return diambil

# Cross Over & Mutation


In [ ]:
def mutation(value,rate):                                                                #input: masukan dan probabilitas
  for i in range(0,len(value)):                                                          #output: hasil mutasi
    if random.random()<=rate:
      if value[i]==0:
        value=value[:i]+"1"+value[i+1:]
      else:
        value=value[:i]+"0"+value[i+1:]
  return value


def buatAnak(a,b,crossRate,mutationRate):                                      #input : array orang tua yang akan di crossover, crossover probability, mutation Probability
    crosspoint=randint(0,len(a))                                                #output : array anak hasi crossover dan mutasi
    a=mutation(a,mutationRate)
    b=mutation(b,mutationRate)

    #cross over
    if random.random() <= crossRate:
      temp=a
      a = a[:crosspoint]+b[crosspoint:]
      b = b[:crosspoint]+temp[crosspoint:]
    return a,b

# Penerusan Generasi

In [ ]:

def getNewPopulation(anak,ortu):                                                          #input : populasi anak dan orangtua
                                                                                          #output : populasi generasi baru
  fitnessAnak=getFitness(anak)
  fitnessOrtu=getFitness(ortu)


  #untuk kemudahan proses perlu untuk menjadikan array kedalam pandas dataframe
  dfPopulasiAnak = pd.DataFrame(anak, 
                    columns=['pertama', 
                              'kedua'])

  dfFitnessAnak = pd.DataFrame(fitnessAnak, 
              columns=['fitness'])

  dfPopulasiAnak=dfPopulasiAnak.join(dfFitnessAnak)

  dfPopulasiOrtu=pd.DataFrame(ortu, 
                      columns=['pertama', 
                                'kedua'])
  
  dfFitnessOrtu=pd.DataFrame(fitnessOrtu, 
              columns=['fitness'])
  
  
  dfPopulasiOrtu=dfPopulasiOrtu.join(dfFitnessOrtu)

  #menggabungkan dataframe orang tua dan anak
  dfPopulasi=pd.concat([dfPopulasiAnak, dfPopulasiOrtu],ignore_index=True)

  #menghapus nilai NA ( karena di fungsi fitness akan diciptakan nila NA jika nilai input diluar batas yang ditentukan)
  dfPopulasi= dfPopulasi.dropna(subset=['fitness'])

  #melakukan pengurutan secara descending
  dfPopulasi = dfPopulasi.sort_values(by=['fitness'],ascending=False,ignore_index=True)

  #menghapus duplicate
  dfPopulasi =dfPopulasi.drop_duplicates(subset=['pertama','kedua'],keep="last")

  #ambil 500 terbaik untuk dijadikan populasi baru
  dfPopulasi =dfPopulasi[:500]

  #memisahkan populasi dengan fitness
  populasi=dfPopulasi.drop(['fitness'],axis=1)

  fitness=dfPopulasi['fitness']

  populasi=populasi.to_numpy()

  fitness=fitness.to_numpy()

  return populasi,fitness


# Program MAIN

In [ ]:
#main


banyakPopulasi=2000
banyakAnak=1000
crossoverRate=0.800
mutationRate=0.3
generasi=300

#masukin populasi inisiasi
population=generateIndividu(banyakPopulasi)

# akan terlihat bahwa  
for gen in range (0,generasi):
  anak=seleksiOrangtua(population,banyakAnak)
  anak=encodeList(anak)

  newAnak=[]
  for kr in anak:
    newAnak.append(buatAnak(kr[0],kr[1],crossoverRate,mutationRate))
  newAnak=decodeList(newAnak)

  population,fitn = getNewPopulation(newAnak,population)

  print(f'generasi ke {gen+1} ')
  print('top 5 :')
  for i in range(0,5):
    print(f'{i+1}. {population[i]} : {fitn[i]}')



generasi ke 1 
top 5 :
1. [0.87636342000000000407 0.99934294000000001290] : 2.48045912824672
2. [1.99827021999999998592 0.98127578000000004188] : 2.4387173668249535
3. [0.98457838842671940860 0.98457838842671940860] : 2.4356824133771453
4. [0.98098755002914450607 0.98098755002914450607] : 2.430993252011642
5. [0.87059063000000003196 0.97143813999999995001] : 2.430104882565952
generasi ke 2 
top 5 :
1. [0.87636342000000000407 0.99934294000000001290] : 2.48045912824672
2. [1.99827021999999998592 0.98127578000000004188] : 2.4387173668249535
3. [0.98457838842671940860 0.98457838842671940860] : 2.4356824133771453
4. [0.98098755002914450607 0.98098755002914450607] : 2.430993252011642
5. [0.87059063000000003196 0.97143813999999995001] : 2.430104882565952
generasi ke 3 
top 5 :
1. [0.87636342000000000407 0.99934294000000001290] : 2.48045912824672
2. [0.99821220897413809325 0.99821220897413809325] : 2.4525481514349
3. [0.99234992688447976050 0.99234992688447976050] : 2.445479118095501
4. [1.998

In [ ]:
[0.99762 0.99762] : 2.451846631980601